<a href="https://colab.research.google.com/github/nisargdoshi9/siamese_network_few_shot_learning/blob/main/Siamese_NW_Fruits360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:

#Configuration environment
import os

os.environ['KAGGLE_USERNAME'] = "nisargx9" # username from the json file
os.environ['KAGGLE_KEY'] = "1e6b4d19094e4587b87ea9e3327aecf9" # key from the json file

In [18]:
cd '/content/drive/MyDrive/Projects/Fruit360'

/content/drive/MyDrive/Projects/Fruit360


In [19]:
!kaggle datasets download -d moltean/fruits

 99% 1.28G/1.28G [00:14<00:00, 141MB/s]
100% 1.28G/1.28G [00:14<00:00, 95.4MB/s]


In [ ]:
# !unzip "/content/drive/MyDrive/Projects/Fruit360/fruits.zip" -d "/content/drive/MyDrive/Projects/Fruit360/fruits"

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import cv2

import tensorflow as tf
import keras


In [29]:
dir = "/content/drive/MyDrive/Projects/Fruit360/fruits/fruits-360_dataset/fruits-360/Training"
split = 0.7
num_files_each_class = 10

folder_list = os.listdir(dir)
print(len(folder_list),": number of categories")

131 : number of categories


In [31]:
x = []
y = []
cat_list = []
y_label = 0
for folder_name in folder_list:
  file_list = os.listdir(dir+'/'+folder_name)
  temp = []
  for file_name in file_list[:num_files_each_class]:
    temp.append(len(x))
    im = cv2.imread(dir+'/'+folder_name+'/'+file_name)
    im = cv2.resize(im,(100,100))
    x.append(im)
    y.append(y_label)
  y_label+=1
  cat_list.append(temp)

cat_list = np.asarray(cat_list)
x = np.asarray(x)/255
y = np.asarray(y)
print('X,y, cat_list shape: ',x.shape,y.shape,cat_list.shape)

X,y shape:  (1310, 100, 100, 3) (1310,) (131, 10)
